In [ ]:
import pandas as pd
import numpy as np

In [ ]:
with open('/content/drive/MyDrive/sentences.txt') as f:
  t=f.readlines()
f.close()
t=np.array(t,dtype='object')
t

array(['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .\n',
       'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "\n',
       'They marched from the Houses of Parliament to a rally in Hyde Park .\n',
       ...,
       'Ilya Kovalchuk scored four goals Sunday with two in each of the first two periods .\n',
       'Russian goaltender Evgeni Nabokov posted consecutive shutout victories over Sweden and Kazakhstan with 48 saves .\n',
       'Although he allowed one goal by Latvia , he faced only seven shots in two periods before being replaced by his back-up .\n'],
      dtype=object)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer=Tokenizer(oov_token='<oov>')
tokenizer.fit_on_texts(t)
vocabsize=len(tokenizer.word_index)+1
encoded_df=tokenizer.texts_to_sequences(t)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len=50
padded_df=pad_sequences(encoded_df,maxlen=max_len,padding='post')
padded_df

array([[  265,     4,   958, ...,     0,     0,     0],
       [ 1180,     4,   163, ...,     0,     0,     0],
       [   46,  1808,    21, ...,     0,     0,     0],
       ...,
       [24095, 24096,  2051, ...,     0,     0,     0],
       [  197, 24097, 24098, ...,     0,     0,     0],
       [  892,    24,   980, ...,     0,     0,     0]], dtype=int32)

In [ ]:
padded_df.shape

(33570, 50)

In [ ]:
with open('/content/drive/MyDrive/labels.txt') as f:
  t1=f.readlines()
f.close()

In [ ]:
d={}
c=1
for i in t1:
  for j in i.split():
    if (j in d):
      pass
    else:
      d[j]=c
      c+=1

In [ ]:
d

{'B-art': 9,
 'B-eve': 15,
 'B-geo': 2,
 'B-gpe': 3,
 'B-nat': 14,
 'B-org': 6,
 'B-per': 4,
 'B-tim': 8,
 'I-art': 10,
 'I-eve': 16,
 'I-geo': 5,
 'I-gpe': 12,
 'I-nat': 17,
 'I-org': 7,
 'I-per': 11,
 'I-tim': 13,
 'O': 1}

In [ ]:
d['I-org']

7

In [ ]:
t2=[[d[i]for i in s.split()]for s in t1]

In [ ]:
max_len=50
t3=pad_sequences(t2,maxlen=max_len,padding='post')
t3

array([[ 1,  1,  1, ...,  0,  0,  0],
       [ 1,  1,  1, ...,  0,  0,  0],
       [ 1,  1,  1, ...,  0,  0,  0],
       ...,
       [ 4, 11,  1, ...,  0,  0,  0],
       [ 3,  1,  4, ...,  0,  0,  0],
       [ 1,  1,  1, ...,  0,  0,  0]], dtype=int32)

In [ ]:
t3.shape

(33570, 50)

In [ ]:
from keras import Model,Input
from keras.layers import Dense,Dropout,Bidirectional,LSTM,Embedding

In [ ]:
input=Input(shape=(max_len))
model=Embedding(vocabsize,output_dim=200,input_length=max_len)(input)
model=Bidirectional(LSTM(128,return_sequences=True))(model)
model=Bidirectional(LSTM(80,return_sequences=True))(model)
model=Bidirectional(LSTM(64,return_sequences=True))(model)
model=Dropout(0.4)(model)
ou=Dense(18,activation='softmax')(model)
model=Model(input,ou)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 200)           4820200   
_________________________________________________________________
bidirectional (Bidirectional (None, 50, 256)           336896    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 160)           215680    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 50, 128)           115200    
_________________________________________________________________
dropout (Dropout)            (None, 50, 128)           0         
_________________________________________________________________
dense (Dense)                (None, 50, 18)            2322  

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(padded_df,t3,epochs=5,batch_size=124)

In [ ]:
s=["raj kumar are working in google "]

In [ ]:
s1=tokenizer.texts_to_sequences(s)

In [ ]:
s1=pad_sequences(s1,maxlen=max_len,padding='post')

In [ ]:
predict=model.predict(s1)

In [ ]:
s2=np.argmax(predict[0],axis=1)

In [ ]:
for i,j in enumerate(s[0].split()):
  print(j,rev_d[s2[i]])

raj B-per
kumar I-per
are O
working O
in O
google B-geo


In [ ]:
d

{'B-art': 9,
 'B-eve': 15,
 'B-geo': 2,
 'B-gpe': 3,
 'B-nat': 14,
 'B-org': 6,
 'B-per': 4,
 'B-tim': 8,
 'I-art': 10,
 'I-eve': 16,
 'I-geo': 5,
 'I-gpe': 12,
 'I-nat': 17,
 'I-org': 7,
 'I-per': 11,
 'I-tim': 13,
 'O': 1}

In [ ]:
rev_d={i:j for j,i in d.items()}